In [54]:
import anndata
import pandas as pd

In [55]:
anndata_obj = anndata.read_h5ad("out_flatten.h5ad", backed="r")

In [56]:
pd.set_option('display.max_columns', None)
anndata_obj.obs

,ROIGroup,ROIGroupCoarse,ROIGroupFine,roi,organism_ontology_term_id,disease_ontology_term_id,self_reported_ethnicity_ontology_term_id,assay_ontology_term_id,sex_ontology_term_id,development_stage_ontology_term_id,donor_id,suspension_type,dissection,fraction_mitochondrial,fraction_unspliced,cell_cycle_score,total_genes,total_UMIs,sample_id,supercluster_term,cluster_id,subcluster_id,cell_type_ontology_term_id,tissue_ontology_term_id,is_primary_data,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage,Supercluster,Supercluster--cell_set_accession,Supercluster--cell_ontology_term_id,Supercluster--cell_ontology_term,Supercluster--rationale,Supercluster--rationale_dois,Supercluster--marker_gene_evidence,Cluster,Cluster--cell_set_accession,Cluster--cell_ontology_term_id,Cluster--cell_ontology_term,Cluster--rationale,Cluster--rationale_dois,Cluster--marker_gene_evidence,Cluster--parent_cell_set_name,Cluster--parent_cell_set_accession,Cluster--user_annotations
CellID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10X362_3:TCAGTGAGTATTGACC,Hypothalamus,Hypothalamus,Hypothalamus,Human MN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000123,H19.30.002,nucleus,Hypothalamus (HTH) - mammillary region of HTH ...,0.000445,0.828378,0.001513,3100.0,11234.0,10X362_3,Oligodendrocyte,50,2996,CL:0000128,UBERON:0001898,False,oligodendrocyte,10x 3' v3,normal,Homo sapiens,male,hypothalamus,European,29-year-old human stage,Oligodendrocyte,CS202210140_469,CL:0000128,oligodendrocyte,,,"PLP1, SOX10",Oligo_50,CS202210140_51,,,,,,Oligodendrocyte,CS202210140_469,
10X362_5:TCCGTGTGTGAAAGTT,Hypothalamus,Hypothalamus,Hypothalamus,Human MN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000136,H19.30.001,nucleus,Hypothalamus (HTH) - mammillary region of HTH ...,0.000153,0.848476,0.002145,2303.0,6527.0,10X362_5,Oligodendrocyte,50,2996,CL:0000128,UBERON:0001898,False,oligodendrocyte,10x 3' v3,normal,Homo sapiens,male,hypothalamus,European,42-year-old human stage,Oligodendrocyte,CS202210140_469,CL:0000128,oligodendrocyte,,,"PLP1, SOX10",Oligo_50,CS202210140_51,,,,,,Oligodendrocyte,CS202210140_469,
10X362_5:CACGGGTAGAGCAGAA,Hypothalamus,Hypothalamus,Hypothalamus,Human MN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000136,H19.30.001,nucleus,Hypothalamus (HTH) - mammillary region of HTH ...,0.000000,0.801024,0.001280,2733.0,7031.0,10X362_5,Oligodendrocyte,50,2996,CL:0000128,UBERON:0001898,False,oligodendrocyte,10x 3' v3,normal,Homo sapiens,male,hypothalamus,European,42-year-old human stage,Oligodendrocyte,CS202210140_469,CL:0000128,oligodendrocyte,,,"PLP1, SOX10",Oligo_50,CS202210140_51,,,,,,Oligodendrocyte,CS202210140_469,
10X362_5:GATTCTTGTATGTCAC,Hypothalamus,Hypothalamus,Hypothalamus,Human MN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000136,H19.30.001,nucleus,Hypothalamus (HTH) - mammillary region of HTH ...,0.000000,0.851632,0.001995,4771.0,22060.0,10X362_5,Oligodendrocyte,50,2998,CL:0000128,UBERON:0001898,False,oligodendrocyte,10x 3' v3,normal,Homo sapiens,male,hypothalamus,European,42-year-old human stage,Oligodendrocyte,CS202210140_469,CL:0000128,oligodendrocyte,,,"PLP1, SOX10",Oligo_50,CS202210140_51,,,,,,Oligodendrocyte,CS202210140_469,
10X362_6:AGGACTTGTATCCTTT,Hypothalamus,Hypothalamus,Hypothalamus,Human MN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000136,H19.30.001,nucleus,Hypothalamus (HTH) - mammillary region of HTH ...,0.000000,0.848084,0.001936,3241.0,12915.0,10X362_6,Oligodendrocyte,50,2996,CL:0000128,UBERON:0001898,False,oligodendrocyte,10x 3' v3,normal,Homo sapiens,male,hypothalamus,European,42-year-old human stage,Oligodendrocyte,CS202210140_469,CL:0000128,oligodendrocyte,,,"PLP1, SOX10",Oligo_50,CS202210140_51,,,,,,Oligodendrocyte,CS202210140_469,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [57]:
anndata_obj.obs["Cluster"].unique().tolist()[:5]

['Oligo_50', 'Oligo_49', 'Oligo_44', 'COP_42', 'COP_43']

In [58]:
anndata_obj.obs[anndata_obj.obs["Cluster--parent_cell_set_name"] == "Vascular"]

,ROIGroup,ROIGroupCoarse,ROIGroupFine,roi,organism_ontology_term_id,disease_ontology_term_id,self_reported_ethnicity_ontology_term_id,assay_ontology_term_id,sex_ontology_term_id,development_stage_ontology_term_id,donor_id,suspension_type,dissection,fraction_mitochondrial,fraction_unspliced,cell_cycle_score,total_genes,total_UMIs,sample_id,supercluster_term,cluster_id,subcluster_id,cell_type_ontology_term_id,tissue_ontology_term_id,is_primary_data,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage,Supercluster,Supercluster--cell_set_accession,Supercluster--cell_ontology_term_id,Supercluster--cell_ontology_term,Supercluster--rationale,Supercluster--rationale_dois,Supercluster--marker_gene_evidence,Cluster,Cluster--cell_set_accession,Cluster--cell_ontology_term_id,Cluster--cell_ontology_term,Cluster--rationale,Cluster--rationale_dois,Cluster--marker_gene_evidence,Cluster--parent_cell_set_name,Cluster--parent_cell_set_accession,Cluster--user_annotations
CellID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10X377_4:CCACGTTAGGAAGTCC,Thalamus,Thalamus,Thalamus,Human LP,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000123,H19.30.002,nucleus,Thalamus (THM) - lateral nuclear complex of th...,0.000808,0.511309,0.000808,791.0,1238.0,10X377_4,Vascular,23,3236,CL:0000669,UBERON:0010225,False,pericyte,10x 3' v3,normal,Homo sapiens,male,thalamic complex,European,29-year-old human stage,Vascular,CS202210140_465,CL:0002139,endothelial cell of vascular tree,,,"ACTA2, CLDN5",Per_23,CS202210140_24,,,,,,Vascular,CS202210140_465,
10X206_4:CATACTTTCTGAGAAA,Thalamus,Thalamus,Thalamus,Human Pul,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000144,H18.30.002,nucleus,Thalamus (THM) - lateral nuclear complex of th...,0.010932,0.652847,0.000825,2282.0,4848.0,10X206_4,Vascular,23,3237,CL:0000669,UBERON:0010225,False,pericyte,10x 3' v3,normal,Homo sapiens,male,thalamic complex,European,50-year-old human stage,Vascular,CS202210140_465,CL:0002139,endothelial cell of vascular tree,,,"ACTA2, CLDN5",Per_23,CS202210140_24,,,,,,Vascular,CS202210140_465,
10X359_4:ATCACTTAGTAGTCCT,Thalamus,Thalamus,Thalamus,Human Pul,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000123,H19.30.002,nucleus,Thalamus (THM) - lateral nuclear complex of th...,0.012555,0.502825,0.000000,1045.0,1593.0,10X359_4,Vascular,23,3236,CL:0000669,UBERON:0010225,False,pericyte,10x 3' v3,normal,Homo sapiens,male,thalamic complex,European,29-year-old human stage,Vascular,CS202210140_465,CL:0002139,endothelial cell of vascular tree,,,"ACTA2, CLDN5",Per_23,CS202210140_24,,,,,,Vascular,CS202210140_465,
10X215_7:GTAGATCAGTTCATCG,Cerebral cortex,Cerebral cortex,Cerebral cortex,Human A23,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000144,H18.30.002,nucleus,Cerebral cortex (Cx) - Caudal cingulate gyrus ...,0.007209,0.570284,0.000801,1456.0,2497.0,10X215_7,Vascular,23,3236,CL:0000669,UBERON:0000956,False,pericyte,10x 3' v3,normal,Homo sapiens,male,cerebral cortex,European,50-year-old human stage,Vascular,CS202210140_465,CL:0002139,endothelial cell of vascular tree,,,"ACTA2, CLDN5",Per_23,CS202210140_24,,,,,,Vascular,CS202210140_465,
10X221_5:ACTTATCGTCAGTCGC,Cerebral cortex,Cerebral cortex,Cerebral cortex,Human STG,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000144,H18.30.002,nucleus,Cerebral cortex (Cx) - Superior Temporal Gyrus...,0.004010,0.661153,0.001504,1103.0,1995.0,10X221_5,Vascular,23,3236,CL:0000669,UBERON:0000956,False,pericyte,10x 3' v3,normal,Homo sapiens,male,cerebral cortex,European,50-year-old human stage,Vascular,CS202210140_465,CL:0002139,endothelial cell of vascular tree,,,"ACTA2, CLDN5",Per_23,CS202210140_24,,,,,,Vascular,CS202210140_465,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [59]:
anndata_obj.obs[anndata_obj.obs["Supercluster"] == "Microglia"]

,ROIGroup,ROIGroupCoarse,ROIGroupFine,roi,organism_ontology_term_id,disease_ontology_term_id,self_reported_ethnicity_ontology_term_id,assay_ontology_term_id,sex_ontology_term_id,development_stage_ontology_term_id,donor_id,suspension_type,dissection,fraction_mitochondrial,fraction_unspliced,cell_cycle_score,total_genes,total_UMIs,sample_id,supercluster_term,cluster_id,subcluster_id,cell_type_ontology_term_id,tissue_ontology_term_id,is_primary_data,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage,Supercluster,Supercluster--cell_set_accession,Supercluster--cell_ontology_term_id,Supercluster--cell_ontology_term,Supercluster--rationale,Supercluster--rationale_dois,Supercluster--marker_gene_evidence,Cluster,Cluster--cell_set_accession,Cluster--cell_ontology_term_id,Cluster--cell_ontology_term,Cluster--rationale,Cluster--rationale_dois,Cluster--marker_gene_evidence,Cluster--parent_cell_set_name,Cluster--parent_cell_set_accession,Cluster--user_annotations
CellID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10X330_3:CATCCCAGTGCCGAAA,Cerebral nuclei,Cerebral nuclei,Basal forebrain,Human SEP,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000123,H19.30.002,nucleus,Basal forebrain (BF) - septal nuclei - SEP,0.000885,0.635586,0.001475,1687.0,3389.0,10X330_3,Microglia,4,3268,CL:0000878,UBERON:8440012,False,central nervous system macrophage,10x 3' v3,normal,Homo sapiens,male,cerebral nuclei,European,29-year-old human stage,Microglia,CS202210140_464,CL:0000129,microglia,,,PTPRC,Mgl_4,CS202210140_5,,,,,,Microglia,CS202210140_464,
10X222_5:GGGACTCAGCCACAAG,Spinal cord,Spinal cord,Spinal cord,Human SpC,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000144,H18.30.002,nucleus,Spinal cord - SpC,0.009020,0.699339,0.001804,876.0,1663.0,10X222_5,Microglia,4,3268,CL:0000878,UBERON:0002240,False,central nervous system macrophage,10x 3' v3,normal,Homo sapiens,male,spinal cord,European,50-year-old human stage,Microglia,CS202210140_464,CL:0000129,microglia,,,PTPRC,Mgl_4,CS202210140_5,,,,,,Microglia,CS202210140_464,
10X390_5:TCTGCCAAGCACGATG,Hindbrain,Pons,Pons,Human PnEN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000123,H19.30.002,nucleus,Pons (Pn) - efferent nuclei of cranial nerves ...,0.000000,0.695973,0.000000,816.0,1490.0,10X390_5,Microglia,4,3268,CL:0000878,UBERON:0000988,False,central nervous system macrophage,10x 3' v3,normal,Homo sapiens,male,pons,European,29-year-old human stage,Microglia,CS202210140_464,CL:0000129,microglia,,,PTPRC,Mgl_4,CS202210140_5,,,,,,Microglia,CS202210140_464,
10X386_2:CGAGAAGCAGGACATG,Hindbrain,Myelencephalon,Myelencephalon,Human MoAN,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000136,H19.30.001,nucleus,Myelencephalon (medulla oblongata) (Mo) - affe...,0.001047,0.720419,0.000524,1125.0,1910.0,10X386_2,Microglia,4,3269,CL:0000878,UBERON:0005290,False,central nervous system macrophage,10x 3' v3,normal,Homo sapiens,male,myelencephalon,European,42-year-old human stage,Microglia,CS202210140_464,CL:0000129,microglia,,,PTPRC,Mgl_4,CS202210140_5,,,,,,Microglia,CS202210140_464,
10X222_6:GAGGGTAAGAGGGTAA,Spinal cord,Spinal cord,Spinal cord,Human SpC,NCBITaxon:9606,PATO:0000461,HANCESTRO:0005,EFO:0009922,PATO:0000384,HsapDv:0000144,H18.30.002,nucleus,Spinal cord - SpC,0.007531,0.665272,0.001255,1433.0,2390.0,10X222_6,Microglia,4,3269,CL:0000878,UBERON:0002240,False,central nervous system macrophage,10x 3' v3,normal,Homo sapiens,male,spinal cord,European,50-year-old human stage,Microglia,CS202210140_464,CL:0000129,microglia,,,PTPRC,Mgl_4,CS202210140_5,,,,,,Microglia,CS202210140_464,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10X194_8:GAAATGAGTTCGGCTG,Midbrain,Midbrain,Midbrain,Human SN,NCBITaxon:9606,PATO:0000461,H

In [60]:
anndata_obj.uns

{'Cluster--rank': '0',
 'Supercluster--rank': '1',
 'author_name': '',
 'batch_condition': array(['sample_id'], dtype=object),
 'schema_version': '3.1.0',
 'title': 'All non-neuronal cells'}